In [186]:
# Lunch Menu Scraper

from urllib import request
from bs4 import BeautifulSoup as bs
from datetime import datetime
import requests
import json

import settings as SETTINGS

class lunchScraper(object):
    
    def __init__(self):
        self.data = {
            "title": "Daily Menu for {}".format(datetime.now().strftime("%A, %d-%b")),
            "body": {
                "menus": []
            }
        }

    def add_menu(self, name, url, selector, n=0):
        try:
            with request.urlopen(url) as response:
                html = response.read()
                soup = bs(html, 'lxml')
                selected = soup.select(selector)
                if n == -1:
                    text = "\n".join([item.get_text() for item in selected])
                elif isinstance(n, range):
                    text = []
                    for i in n:
                        text.append( clean(selected[i].get_text()) )
                    text = "\n".join(text)
                else:
                    text = selected[n].get_text()
                text = [t for t in text.split("\n") if len(t) > 0]
            self.data['body']['menus'].append({'name':name,'url':url,'menu':text})
        except Exception as e:
            print("Couldn't find menu for {}. Error: {}".format(name, e))

    def send_message(self):
        auth = ("api", SETTINGS.MAIL_API_KEY)
        data = {
            "from": SETTINGS.FROM,
            "to": SETTINGS.TO,
            "subject": self.data['title'],
            "h:X-Mailgun-Variables": json.dumps(self.data),
            "t:text": "yes",
            "template": "daily-menu",
        }
        r = requests.post(SETTINGS.MAIL_URL, auth=auth, data=data)
        return r
    
    def get_recipients(self):
        with open(SETTINGS.SUBSCRIBERS, 'r') as f:
            subscribers = json.load(f)
            recipients = []
            for subscriber in subscribers:
                if subscriber['verified']:
                    recipients.append(subscriber['email'])
        return recipients
    
def your_restaurants(temp):

#     # Pastva
#     name = "Pastva"
#     url = "https://www.pastva-restaurant.cz/nase-menu/"
#     selector = "#ffe_widget-2 .eff-panel-body"
#     temp.add_menu(name, url, selector)

#     # Sodexo
#     name = "Sodexo, Riverview"
#     url = "http://riverview.extranet.prod.dator3.cz/en/menu-for-the-week/"
#     weekday = 4 - datetime.today().weekday()
#     weekday = weekday if weekday > 0 else 0
#     tag = "#menu-{} .popisJidla".format(str(weekday))
#     selector = tag
#     temp.add_menu(name, url, selector, n=-1)

#     # FIVE - Weekly
#     name = "Dave B, Five (Week)"
#     url = "https://www.daveb.cz/cs/denni-nabidka"
#     selector = ".article .row div"
#     temp.add_menu(name, url, selector, n=1)

#     # FIVE - Today
#     name = "Dave B, Five (Today)"
#     url = "https://www.daveb.cz/cs/denni-nabidka"
#     selector = ".article .row div"
#     weekday = 3 + datetime.today().weekday()
#     weekday = weekday if int(weekday) <= 7 else 3
#     temp.add_menu(name, url, selector, n=weekday)
    
#     # Potrefena Husa - Na Verandach
#     name = "Potrefena Husa - Na Verandach"
#     url = "https://www.phnaverandach.cz/"
#     selector = ".listek-out .listek #table-1 .food-title"
#     temp.add_menu(name, url, selector, n=-1)

    # U Kroka
#     name = "U Kroka"
#     url = "https://www.ukroka.cz/menus-1"
#     selector = ".menuApp"
#     temp.add_menu(name, url, selector)

    # Lavande Restaurant - Weekly
    name = "Lavande Restaurant - Week"
    url = "https://restaurantlavande.cz/menu/#week-menu"
    selector = ".week-menu__header ~ .menus .menus__menu-content h3 ~ div .food__name"
    temp.add_menu(name, url, selector, n=range(0,3))

    # Lavande Restaurant - Daily
    name = "Lavande Restaurant - Week"
    url = "https://restaurantlavande.cz/menu/#week-menu"
    weekday = datetime.today().weekday()
    weekday = weekday if weekday < 5 else 4
    n = (( weekday + 1) * 4) - 1
    selector = ".week-menu__header ~ .menus .menus__menu-content h3 ~ div .food__name"
    temp.add_menu(name, url, selector, n=range(n,n+4))

# if __name__ == "__main__":
#     x = lunchScraper()
#     your_restaurants(x)
#     x.send_message()

x = lunchScraper()
# your_restaurants(x)
x.get_recipients()
# x.send_message()

['kotek.vojtech@gmail.com']

In [185]:
x.data

{'body': {'menus': [{'menu': ['Palačinka s pomerančovým likérem a karamelem',
     'Domácí rybízová limonáda',
     'Grilovaný rib eye steak s bylinkovým máslem restované mini brambory s červenou cibulí pepřová omáčka'],
    'name': 'Lavande Restaurant - Week',
    'url': 'https://restaurantlavande.cz/menu/#week-menu'},
   {'menu': ['Francouzská cibulačka GF DF se sýrovými krutony',
     'Kachní soté s restovaným pak choi karotkou a sezamem rýžové nudle',
     'Kuřecí karbanátek s uzeným sýrem šťouchané brambory s pórkem a smetanou',
     'Cizrnovorajčatové karí s čerstvým kozím sýrem pečené mini brambory'],
    'name': 'Lavande Restaurant - Week',
    'url': 'https://restaurantlavande.cz/menu/#week-menu'}]},
 'title': 'Daily Menu for Monday, 10-Jun'}

In [175]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
import re

def add_menu(name, url, selector, n=0):
    try:
        with request.urlopen(url) as response:
            html = response.read()
            soup = bs(html, 'lxml')
            selected = soup.select(selector)
            if n == -1:
                text = "\n".join([item.get_text() for item in selected])
            elif isinstance(n, range):
                text = []
                for i in n:
                    text.append( clean(selected[i].get_text()) )
                text = "\n".join(text)
            else:
                text = selected[n].get_text()
            # Clean the text up.
            text = [t for t in text.split("\n") if len(t) > 0]
        return {'name':name,'url':url,'menu':text}
    except Exception as e:
        print("Couldn't find menu for {}. Error: {}".format(name, e))
        

# Lavande Restaurant - Weekly
name = "Lavande Restaurant - Week"
url = "https://restaurantlavande.cz/menu/#week-menu"
selector = ".week-menu__header ~ .menus .menus__menu-content h3 ~ div .food__name"
z = add_menu(name, url, selector, n=range(0,3))

# Lavande Restaurant - Daily
name = "Lavande Restaurant - Daily"
url = "https://restaurantlavande.cz/menu/#week-menu"
weekday = datetime.today().weekday()
weekday = weekday if weekday < 5 else 4
n = (( weekday + 1) * 4) - 1
selector = ".week-menu__header ~ .menus .menus__menu-content h3 ~ div .food__name"

z = add_menu(name, url, selector, n=range(n,n+4))

pp(z)

{'menu': ['Francouzská cibulačka GF DF se sýrovými krutony',
          'Kachní soté s restovaným pak choi karotkou a sezamem rýžové nudle',
          'Kuřecí karbanátek s uzeným sýrem šťouchané brambory s pórkem a '
          'smetanou',
          'Cizrnovorajčatové karí s čerstvým kozím sýrem pečené mini brambory'],
 'name': 'Lavande Restaurant - Week',
 'url': 'https://restaurantlavande.cz/menu/#week-menu'}


In [107]:
r = request.urlopen(url)
soup = bs(r.read(), 'lxml')

In [155]:
selector = ".week-menu__header ~ .menus .menus__menu-content h3:nth-of-type(1) ~ div .food__name"
z = soup.select(selector)[0]
print(z)

def clean(string):
    return "".join([char for char in string if char.isalnum() or char == " "])

<div class="food__name">
											Palačinka s pomerančovým likérem a karamelem
										</div>


In [125]:
clean(z)

'Domácírybízoválimonáda'

In [136]:
x = range(2,5)
isinstance(x, range)

True

In [ ]:
M = 4
T = 8
W = 12
T = 16
F = 20